In [14]:
API_OPEN_DATA_TIMEOUT = 5
API_OPEN_DATA_RETRIES = 3
API_OPENDATA_DATASET = 'conso-elec-gaz-annuelle-par-secteur-dactivite-agregee-iris'
API_OPENDATA_SERVER = 'https://opendata.agenceore.fr/api'
DB_OLD_DATE = '1975-01-01'
PLOT_QUANTILE  =(0,0.1,0.75,0.9,0.98,0.99,0.995)
YEAR_ENERGY_API  = 2019

ENERGY_NEW_INDIVIDUAL = 23.5
ENERGY_OLD_INDIVIDUAL = 26.0
ENERGY_NEW_BUILDING = 13.0
ENERGY_OLD_BUILDING = 17.0

TOPO_DATE = '2020-12-15'
TOPO_VER = 'BDTOPO_3-0_'
TOPO_TYPE = 'TOUSTHEMES_SHP_LAMB93'

IRIS_DATE = '2020-01-01'
IRIS_VER = 'IRIS-GE_2-0_'
IRIS_TYPE = 'SHP_LAMB93'

DB_NAME = "Energy"
DB_SCHEMA = "public"
DB_HOST = "localhost"
DB_USER = "postgres"
DB_PASSWORD = "postgres" #TODO hide the password
DB_PORT = "5432"

FTP_TOPO_SERVER = 'ftp3.ign.fr'
FTP_TOPO_USER = 'BDTOPO_V3_NL_ext'
FTP_TOPO_PASSWORD = 'Ohp3quaz2aideel4'

FTP_IRIS_SERVER = 'ftp3.ign.fr'
FTP_IRIS_USER = 'Iris_GE_ext'
FTP_IRIS_PASSWORD = 'eeLoow1gohS1Oot9'


INPUT_DIR = 'input'

In [15]:
import psycopg2
from collections import defaultdict
from datetime import datetime,date
from constants import DB_OLD_DATE,YEAR_ENERGY_API,DB_HOST, DB_USER, DB_PASSWORD,ENERGY_NEW_INDIVIDUAL,ENERGY_OLD_INDIVIDUAL,ENERGY_NEW_BUILDING,ENERGY_OLD_BUILDING

from open_data import IrisConsumption

import shapely
import shapely.wkt


class Batiment(object):

    def __init__(self, db_name, dept, host = DB_HOST, user = DB_USER, password = DB_PASSWORD):

        self.db_name = db_name
        self.host = host
        self.user = user
        self.dept = dept

        # TODO : password not in clear text
        self.password = password

        self.conn = psycopg2.connect(
            host=self.host,
            database=self.db_name,
            user=self.user,
            password=self.password)


    def get_insee(self):
        cur = self.conn.cursor()
        str_sql = f"""
            select distinct insee_com,nom_com from public."iris_ge_{self.dept}"
            """
        cur.execute(str_sql)
        rows = cur.fetchall()
        return rows

    def _get_batiments(self):
        # create a cursor
        cur = self.conn.cursor()

        str_sql = f"""
            select iris.code_iris,
		    bat.date_app,
		    bat.nb_logts,
            ST_AsText(ST_Force2D(bat.geom)),
            bat.id,
            iris.insee_com,
            iris.nom_com
            from public."batiment_{self.dept}" as bat
            JOIN public."iris_ge_{self.dept}" AS iris
            ON ST_Contains(iris.geom, bat.geom)
            where (usage1 = 'Résidentiel' or usage2 = 'Résidentiel' or usage1 = 'RÃ©sidentiel' or usage2 = 'RÃ©sidentiel') and nb_logts > 0
            """

        cur.execute(str_sql)
        rows = cur.fetchall()

        list_batiments = [
            {
                'id'        : row[4],
                'iris'      : row[0],
                'insee_com' : row[5],
                'nom_com'   : row[6],
                'date_app'  : row[1],
                'nb_housing'  : row[2],
                'geometry'      : shapely.wkt.loads(row[3]),
            } for row in rows
        ]

        cur.close()

        return list_batiments


    def housing_by_iris2(self):
        self.list_batiments = self._get_batiments()

        old_date = date.fromisoformat(DB_OLD_DATE)
        bat_dispatch = defaultdict(lambda: defaultdict(int))

        for i,bat in enumerate(self.list_batiments):
            if bat['date_app'] is None:
                date_app = old_date
            else:
                date_app = bat['date_app']
            iris = bat['iris']
            if bat['nb_housing'] == 1:
                if date_app > old_date:
                    bat_dispatch[iris]['h_new'] +=1
                    self.list_batiments[i]['type'] = 'h_new'
                else:
                    bat_dispatch[iris]['h_old'] += 1
                    self.list_batiments[i]['type'] = 'h_old'
            elif date_app > old_date:
                    bat_dispatch[iris]['c_new'] += bat['nb_housing']
                    self.list_batiments[i]['type'] = 'c_new'
            else:
                    bat_dispatch[iris]['c_old'] += bat['nb_housing']
                    self.list_batiments[i]['type'] = 'c_old'
        return bat_dispatch



    def get_batiments_consumption(self):

        consumption_by_housing_type = {
            'h_new' : ENERGY_NEW_INDIVIDUAL,
            'h_old' : ENERGY_OLD_INDIVIDUAL,
            'c_new' : ENERGY_NEW_BUILDING,
            'c_old' : ENERGY_OLD_BUILDING,
        }

        iris_consumption = IrisConsumption(self.dept,YEAR_ENERGY_API)
        dict_iris_consumption = iris_consumption.consumption_by_iris()

        dict_dispatch = self.housing_by_iris2()
        for iris_code,dispatch in dict_dispatch.items():
            dict_dispatch[iris_code]['ratio'] = ratio_for_iris(dispatch,consumption_by_housing_type,dict_iris_consumption[iris_code])


        for i,bat in enumerate(self.list_batiments):
            consumption = round(consumption_by_housing_type[bat['type']] * dict_dispatch[bat['iris']]['ratio'],2)
            self.list_batiments[i]['consumption'] = consumption

        return self.list_batiments

def ratio_for_iris(dispatch,consumption_by_housing_type,iris_consumption):
    total_theo_consumption = 0
    for key,number_housing in dispatch.items():
        total_theo_consumption += number_housing * consumption_by_housing_type[key]
    if total_theo_consumption != 0:
        return iris_consumption / total_theo_consumption
    else:
        return 0


base = Batiment("Energy",75)
print(base.get_insee())

[('75106', 'Paris 6e Arrondissement'), ('75110', 'Paris 10e Arrondissement'), ('75104', 'Paris 4e Arrondissement'), ('75103', 'Paris 3e Arrondissement'), ('75108', 'Paris 8e Arrondissement'), ('75118', 'Paris 18e Arrondissement'), ('75115', 'Paris 15e Arrondissement'), ('75120', 'Paris 20e Arrondissement'), ('75119', 'Paris 19e Arrondissement'), ('75101', 'Paris 1er Arrondissement'), ('75116', 'Paris 16e Arrondissement'), ('75112', 'Paris 12e Arrondissement'), ('75105', 'Paris 5e Arrondissement'), ('75109', 'Paris 9e Arrondissement'), ('75117', 'Paris 17e Arrondissement'), ('75114', 'Paris 14e Arrondissement'), ('75113', 'Paris 13e Arrondissement'), ('75107', 'Paris 7e Arrondissement'), ('75102', 'Paris 2e Arrondissement'), ('75111', 'Paris 11e Arrondissement')]


In [20]:
def get_config():
    

    return {
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
        {
          "id": "su1dkk",
          "type": "geojson",
          "config": {
            "dataId": "Energy_Consumption",
            "label": "Energy Consumption",
            "color": [
              18,
              147,
              154
            ],
            "columns": {
              "geojson": "geometry"
            },
            "isVisible": True,
            "visConfig": {
              "opacity": 0.8,
              "strokeOpacity": 0.8,
              "thickness": 0.5,
              "strokeColor": [
                221,
                178,
                124
              ],
              "colorRange": {
                "name": "Uber Viz Diverging 1.5",
                "type": "diverging",
                "category": "Uber",
                "colors": [
                  "#00939C",
                  "#5DBABF",
                  "#BAE1E2",
                  "#F8C0AA",
                  "#DD7755",
                  "#C22E00"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radius": 10,
              "sizeRange": [
                0,
                10
              ],
              "radiusRange": [
                0,
                50
              ],
              "heightRange": [
                0,
                500
              ],
              "elevationScale": 5,
              "stroked": False,
              "filled": True,
              "enable3d": False,
              "wireframe": False
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": {
              "name": "consumption",
              "type": "real"
            },
            "colorScale": "quantile",
            "sizeField": None,
            "sizeScale": "linear",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "heightField": None,
            "heightScale": "linear",
            "radiusField": None,
            "radiusScale": "linear"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "Energy_Consumption": [
              {
                "name": "id",
                "format": None
              },
              {
                "name": "consumption",
                "format": None
              },
              {
                "name": "nb_housing",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": 48.847934840838576,
      "longitude": 2.346600107794568,
      "pitch": 0,
      "zoom": 10.678695367388373,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}

In [21]:
from keplergl import KeplerGl
import matplotlib.pyplot as plt
import matplotlib


import geopandas as gpd
import pandas as pd

from database import Batiment

from constants import PLOT_QUANTILE
import os


def bat_plot(list_batiments):


    gpd_consumption = gpd.GeoDataFrame(list_batiments)
    
    gpd_consumption.crs = {'init' :'epsg:4326'}

    gpd_data = gpd_consumption[['id','geometry','consumption',"nb_housing"]]

    #gpd_data['toto'] = gpd_data.geometry.apply(lambda x: shapely.wkt.dumps(x))

    #print(gpd_data.head())


    map2=KeplerGl()
    map2.add_data(gpd_data,name="Energy_Consumption")
    #display(map2)
    config = get_config()
    map2.save_to_html(file_name='EnergyConsumption_75_92_93_94.html',config = config)


list_dept = [75,92,93,94]

list_batiments = list()
for dept in list_dept:
    batiment = Batiment(DB_NAME,dept)
    list_batiments.extend(batiment.get_batiments_consumption())
        
bat_plot(list_batiments)
            


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to EnergyConsumption_75_92_93_94.html!


In [17]:
"""
https://opendata.agenceore.fr/api/records/1.0/search/?dataset=conso-elec-gaz-annuelle-par-secteur-dactivite-agregee-iris&q=&rows=10000&facet=operateur&facet=annee&facet=filiere&facet=libelle_commune&facet=code_departement&facet=libelle_region&refine.code_departement=75&refine.annee=2019
"""

from constants import API_OPEN_DATA_TIMEOUT, API_OPEN_DATA_RETRIES,API_OPENDATA_DATASET,API_OPENDATA_SERVER
import requests
import os
from collections import defaultdict

class IrisConsumption(object):

    def __init__(self, dept, year, dataset = API_OPENDATA_DATASET, server = API_OPENDATA_SERVER ):
        """initialise the Open_data class

        Parameters
        ----------
        dept : int
            department number
        year : int
            year for the data
        """

        self.dept = dept
        self.year = year
        self.dataset = dataset

        self.server = server
        if not self.server:
            raise ValueError("No osrm server given")

        self.timeout = API_OPEN_DATA_TIMEOUT

        # Session definition
        self.session = requests.Session()
        self.session.mount(
            "https://",
            requests.adapters.HTTPAdapter(
                max_retries=API_OPEN_DATA_RETRIES
            ),
        )

    def _build_base_request(self):

        base_request = "records/1.0/search/?dataset={}&q=&rows=10000&facet=annee&facet=libelle_commune&facet=code_departement&facet=libelle_region&refine.code_departement={}&refine.annee={}".format(
                        self.dataset, self.dept,self.year
        )

        return os.path.join(self.server, base_request)

    def _get_result(self):

        request_url = self._build_base_request()

        try:
            response = self.session.get(request_url, timeout=self.timeout)

            if response.status_code != requests.codes.ok:
                print(f"error in requestiong data : {response.text}")
                return None

            result = response.json()
            return result


        except (IndexError, ValueError):
            return None

    def _get_records(self):

        return self._get_result()['records']

    def get_fields(self):

        records = self._get_records()

        return [record['fields'] for record in records]

    def consumption_by_iris(self):

        fields = self.get_fields()
        consumption=defaultdict(int)
        for field in fields:
            consumption[field['code_iris']] += field['consor']
        return consumption

energy = IrisConsumption(75,2019)
print(energy.consumption_by_iris())


defaultdict(<class 'int'>, {'751010206': 2763.06106814209, '751020501': 7217.79337187269, '751020601': 8888.05995708516, '751020602': 7263.90567422345, '75102XXXX': 0.0, '751030902': 9640.420960948311, '751031001': 11259.71227406156, '751031003': 13441.05876906613, '751031202': 10008.83011320016, '751041401': 12828.42966671453, '751041603': 738.157276318884, '751051702': 8452.48967476142, '751051803': 10419.10352094772, '751051809': 69.3742988718061, '751051901': 14227.81198930429, '751051906': 10290.291498228471, '751052001': 9749.4731187744, '751052005': 675.8276399939759, '751062308': 15803.22760415872, '751062310': 16389.90668176887, '75106XXXX': 0.0, '751072502': 12004.83908587043, '751072705': 14619.40268930394, '751072804': 16764.5835631989, '751072809': 12870.326488080129, '751072812': 0.0, '751082901': 15507.489969081049, '751082902': 10191.66937592592, '751082999': 0.0, '751083003': 5595.18161218, '751083004': 22909.741025465468, '751083102': 5088.34683080609, '751083105': 10

In [ ]:
from database import Batiment


from plotv2 import bat_plot
from index import createHTML
from constants import DB_NAME

import argparse


if __name__ == '__main__':

    parser = argparse.ArgumentParser(description="Generate html files for each insee cod in Paris and aound department")

    parser.add_argument('-d', '--department', action='store',
                        dest='list_dept', type=int, nargs='*',
                        help="Examples: -d 75 92 93 94. "
                        "List of departments to get map for")

    opts = parser.parse_args()

    list_dept = opts.list_dept

    for dept in list_dept:
            batiment = Batiment(DB_NAME,dept)
            list_batiments = batiment.get_batiments_consumption()
            insee = batiment.get_insee()
            bat_plot(list_batiments,insee,dept)

    createHTML(list_dept)